In [ ]:
import re
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import Chrome
import pandas as pd
import time

In [ ]:
def get_tweet_data(card):
    """Extract data from tweet card"""
    username = card.find_element_by_xpath('.//span').text
    try:
        handle = card.find_element_by_xpath('.//span[contains(text(), "@")]').text
    except NoSuchElementException:
        return
    
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except NoSuchElementException:
        return
    
    comment = card.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = card.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text

    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt)
    return tweet  

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

######### Open chrome
coin = input('Coin: ')
date_from = input('Date from: ')
date_until = input('Date until: ')


driver = webdriver.Chrome(ChromeDriverManager().install())
#date_until = '2017-02-01'
driver.get(f'https://twitter.com/search?q={coin}%20min_faves%3A500%20min_retweets%3A0%20until%3A{date_until}%20since%3A{date_from}&src=typed_query&f=live')

# get all tweets on the page
data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")

scrolling = True
count = 0

while scrolling:
    page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
    print(len(page_cards))
    
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)

        count += len(page_cards)
    if count > 10000:
        driver.get(f'https://twitter.com/search?q=f{coin}%20min_faves%3A500%20min_retweets%3A0%20until%3A{data[-1][2][:10]}%20since%3A2017-01-01&src=typed_query&f=live')
        print('Refreshed')
        time.sleep(10)
        count = 0

    scroll_attempt = 0
    while True:
        # check scroll position
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1

            # end of scroll region
            if scroll_attempt >= 20:
                print('attempts exceeded')
                scrolling = False
                break
            else:
                sleep(3) # attempt another scroll
        else:
            last_position = curr_position
            'just break'
            break

# close the web driver
driver.close

In [ ]:
df = pd.DataFrame(data)

df = df.rename(columns={df.columns[0]: 'Username'})
df = df.rename(columns={df.columns[1]: 'Handle'})
df = df.rename(columns={df.columns[2]: 'Date'})
df = df.rename(columns={df.columns[3]: 'Text'})
df = df.rename(columns={df.columns[4]: 'Reply_cnt'})
df = df.rename(columns={df.columns[5]: 'Retweet_cnt'})
df = df.rename(columns={df.columns[6]: 'Like_cnt'})

In [ ]:
df.to_excel(r'')